# Using climate_tutorial:

In [1]:
from pathlib import Path

path = Path("/u/scratch/s/se0ngbin/weatherbench")   # replace with path

In [2]:
import xarray as xr

In [5]:
import sys
sys.path.insert(0, '/u/home/s/se0ngbin/climate_tutorial')

['/u/home/s/se0ngbin/climate_tutorial',
 '../climate_tutorial',
 '/u/home/s/se0ngbin/climate-models/weatherbench-replication',
 '/u/home/s/se0ngbin/.conda/envs/climate-tools/lib/python310.zip',
 '/u/home/s/se0ngbin/.conda/envs/climate-tools/lib/python3.10',
 '/u/home/s/se0ngbin/.conda/envs/climate-tools/lib/python3.10/lib-dynload',
 '',
 '/u/home/s/se0ngbin/.conda/envs/climate-tools/lib/python3.10/site-packages',
 '/u/home/s/se0ngbin/.conda/envs/climate-tools/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg']

In [6]:
path = path/"data/weatherbench/era5/5.625"

In [7]:
from climate_tutorial.utils.data import load_dataset, view

t_dataset = load_dataset(path/"temperature_850").drop("level")
z_dataset = load_dataset(path/"geopotential_500").drop("level")
valid_data = xr.merge([t_dataset, z_dataset])
valid_data # view(valid_data) displays valid_data.t2m so it doesn't work

# had to run the following to get this to work:
# !pip install netCDF4 
# !pip install h5netcdf
# !pip install dask

<xarray.Dataset>
Dimensions:  (lon: 64, lat: 32, time: 350640)
Coordinates:
  * lon      (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * time     (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
Data variables:
    t        (time, lat, lon) float32 dask.array<chunksize=(8760, 32, 64), meta=np.ndarray>
    z        (time, lat, lon) float32 dask.array<chunksize=(8760, 32, 64), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2019-11-21 05:02:47 GMT by grib_to_netcdf-2.14.0: /opt/ecmw...

In [16]:
valid_data = valid_data.isel(time=slice(0, None, 6))

In [9]:
??DataModule

In [7]:
from climate_tutorial.utils.datetime import Year, Days, Hours
from climate_tutorial.data import DataModule

data_module = DataModule(
    dataset = "ERA5",
    task = "forecasting",
    root_dir = path,
    in_vars = ["geopotential_500", "temperature_850"],
    out_vars = ["geopotential_500", "temperature_850"],
    train_start_year = Year(1979),
    val_start_year = Year(2015),
    test_start_year = Year(2017),
    end_year = Year(2018),
    pred_range = Days(3),
    subsample = Hours(6),
    batch_size = 128,
    num_workers = 1
)

Creating train dataset


  0%|          | 0/36 [00:00<?, ?it/s]

KeyError: 'geopotential_500'

In [11]:
from climate_tutorial.models import load_model

model_kwargs = {
    "img_size": [32, 64],
    "patch_size": 2,
    "drop_path": 0.1,
    "drop_rate": 0.1,
    "learn_pos_emb": True,
    "in_vars": data_module.hparams.in_vars,
    "out_vars": data_module.hparams.out_vars,
    "embed_dim": 128,
    "depth": 8,
    "decoder_depth": 0,
    "num_heads": 4,
    "mlp_ratio": 4,
}

optim_kwargs = {
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": 1,
    "max_epochs": 5,
}

model_module = load_model(name = "vit", task = "forecasting", model_kwargs = model_kwargs, optim_kwargs = optim_kwargs)
# model_module = load_model(name = "resnet", task = "forecasting", model_kwargs = model_kwargs, optim_kwargs = optim_kwargs)
# # model_module = load_model(name = "unet", task = "forecasting", model_kwargs = model_kwargs, optim_kwargs = optim_kwargs)

NameError: name 'data_module' is not defined

In [ ]:
# add_description
from climate_tutorial.models import set_climatology
set_climatology(model_module, data_module)

In [ ]:
from climate_tutorial.training import Trainer, WandbLogger

trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)

In [ ]:
trainer.fit(model_module, data_module)

In [ ]:
trainer.test(model_module, data_module)